In [1]:
import ast
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit
from tqdm.auto import tqdm

DAY = 15
YEAR = 2022

In [2]:
# use test data
raw_test = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3


In [3]:
def parse_data(data):
    data = data.split("\n")
    data = [np.array(re.findall("-{0,1}[\d]+", d), dtype=int) for d in data]
    data = {(sx, sy): (bx, by) for sx, sy, bx, by in data}
    return data


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy

{(2, 18): (-2, 15),
 (9, 16): (10, 16),
 (13, 2): (15, 3),
 (12, 14): (10, 16),
 (10, 20): (10, 16),
 (14, 17): (10, 16),
 (8, 7): (2, 10),
 (2, 0): (2, 10),
 (0, 11): (2, 10),
 (20, 14): (25, 17),
 (17, 20): (21, 22),
 (16, 7): (15, 3),
 (14, 3): (15, 3),
 (20, 1): (15, 3)}

# Part 1

In [4]:
def mdist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])


def unify_ranges(ranges):
    ranges = sorted(map(list, ranges))

    new_ranges = [ranges[0]]
    for cs, ce in ranges[1:]:
        ps, pe = new_ranges[-1]
        if cs <= pe:
            new_ranges[-1][1] = max(ce, pe)
        else:
            new_ranges.append([cs, ce])

    return new_ranges

In [5]:
def find_for_y(data, distances, bs_list, y):
    non_eligible_ranges = []
    for ((sx, sy), (bx, by)), max_dist in zip(data.items(), distances):
        if abs(sy - y) > max_dist:
            continue

        wiggle_room = max_dist - abs(sy - y)
        x1, x2 = sx - wiggle_room, sx + wiggle_room
        non_eligible_ranges.append((x1, x2 + 1))

    bs_count = len([bs for bs in bs_list if bs[1] == y])
    ranges = unify_ranges(non_eligible_ranges)
    result = sum([x2 - x1 for x1, x2 in ranges]) - bs_count
    return result


data = real.copy()

distances = [mdist(s, b) for s, b in data.items()]
y = 10 if (2, 18) in data else 2000000
bs_list = {el for pair in data.items() for el in pair}

result = find_for_y(data, distances, bs_list, y)
result

5403290

In [6]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [7]:
def find_for_y_v2(data, distances, y, lim):
    non_eligible_ranges = []
    for ((sx, sy), (bx, by)), max_dist in zip(data.items(), distances):
        if abs(sy - y) > max_dist or sx + max_dist < 0 or sx - max_dist > lim:
            continue

        wiggle_room = max_dist - abs(sy - y)
        x1, x2 = sx - wiggle_room, sx + wiggle_room
        non_eligible_ranges.append((x1, x2 + 1))

    ranges = unify_ranges(non_eligible_ranges)
    if len(ranges) > 1:
        return freq(ranges[0][1], y)


def freq(x, y):
    return x * 4000000 + y


data = real.copy()

distances = [mdist(s, b) for s, b in data.items()]
lim = 20 if (2, 18) in data else 4000000

for y in range(0, lim + 1):
    result = find_for_y_v2(data, distances, y, lim)
    if result is not None:
        break

result

10291582906626

In [ ]:
# submit(result, part="b", day=DAY, year=YEAR)